#### How are we going yo proceed to get our data?

#### First we import required libraries

In [1]:
# @Hidden_cell

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
# @Hidden_cell

!pip install folium
import pandas as pd
import folium

    100% |████████████████████████████████| 92kB 7.2MB/s eta 0:00:01
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/

#### Then we get our City Coordinates

In [3]:
# @Hidden_cell

address = 'Abidjan'

geolocator = Nominatim(user_agent="Abidjan_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Abidjan are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Abidjan are 5.4091179, -4.0422099.


#### We then use our Foursquare API credentials to generate an URL 

In [7]:
# @Hidden_cell

CLIENT_ID='WGIIPIREBQP0LGQDHPA0BUDOCMI425BZHPJHM40XDS0NKHWR'
CLIENT_SECRET='CQIDUJYNZUJBTNS3IVRQQF1KQBC23L3YWATDCSGPFOVHOU4L'
VERSION='20180605'

In [14]:
# @Hidden_cell

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 50000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WGIIPIREBQP0LGQDHPA0BUDOCMI425BZHPJHM40XDS0NKHWR&client_secret=CQIDUJYNZUJBTNS3IVRQQF1KQBC23L3YWATDCSGPFOVHOU4L&v=20180605&ll=5.4091179,-4.0422099&radius=50000&limit=100'

#### After that, we send the request and examine results

In [15]:
# @Hidden_cell

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c936a9c9fb6b73b7093af92'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4e4b07a018503d60ab7f47fa-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d14e941735',
         'name': 'American Restaurant',
         'pluralName': 'American Restaurants',
         'primary': True,
         'shortName': 'American'}],
       'id': '4e4b07a018503d60ab7f47fa',
       'location': {'address': 'Rue J81',
        'cc': 'CI',
        'city': 'Abidjan',
        'country': "Côte d'Ivoire",
        'crossStreet': 'at Rue J50',
        'distance': 7515,
        'formattedAddress': ['Rue J81 (at Rue J50) Abidjan', "Côte d'Ivoire"],
        'labeledLatLngs': [{'label': 'display',
  

In [16]:
# @Hidden_cell

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
# @Hidden_cell

#Let's structure our results into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Norima,American Restaurant,5.363668,-3.992067
1,Hollywood Boulevard,Lounge,5.375848,-3.990464
2,V.I.P Discothèque,Nightclub,5.389026,-3.992774
3,Le Gelato,Ice Cream Shop,5.401203,-3.978239
4,Des Gateaux & Du Pain,Bakery,5.360270,-3.989671


#### We get the data and display them in a dataframe with the following properties: Name, category, Latitude and longitude

In [23]:
# @Hidden_cell

#Number of venues returned by FourSquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

54 venues were returned by Foursquare.


#### here is the full Dataset we get from our request

In [22]:
# @Hidden_cell

nearby_venues

,name,categories,lat,lng
0,Norima,American Restaurant,5.363668,-3.992067
1,Hollywood Boulevard,Lounge,5.375848,-3.990464
2,V.I.P Discothèque,Nightclub,5.389026,-3.992774
3,Le Gelato,Ice Cream Shop,5.401203,-3.978239
4,Des Gateaux & Du Pain,Bakery,5.360270,-3.989671
5,Bao Café,Coffee Shop,5.348778,-3.996881
6,Cap Nord,Shopping Mall,5.355723,-3.966520
7,Sofitel Abidjan Hôtel Ivoire,Hotel,5.326445,-4.004526
8,Hypermarché SOCOCE,Shopping Mall,5.373198,-3.998634
9,Ivoire Golf Club,Golf Course,5.336301,-3.962488


In [29]:
# @Hidden_cell

# create map of New York using latitude and longitude values
map_abidjan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_abidjan)  
    
map_abidjan

#### We then group the venues by category to have an idea of which activity we can have in our park

In [30]:
# @Hidden_cell

#Grouping Venues
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
African Restaurant,5,5,5
American Restaurant,1,1,1
BBQ Joint,1,1,1
Bakery,2,2,2
Bar,1,1,1
Basketball Stadium,1,1,1
Beach,1,1,1
Beach Bar,1,1,1
Boutique,1,1,1


In [31]:
# @Hidden_cell

#Number of unique categories returned
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 34 uniques categories.


#### We then use one hot encoding to display data categories by columns

In [32]:
# @Hidden_cell

# one hot encoding
Abidjan_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")
Abidjan_onehot.head()

,African Restaurant,American Restaurant,BBQ Joint,Bakery,Bar,Basketball Stadium,Beach,Beach Bar,Boutique,Brewery,Café,Cocktail Bar,Coffee Shop,Fast Food Restaurant,Frozen Yogurt Shop,Golf Course,Halal Restaurant,Hotel,Ice Cream Shop,Italian Restaurant,Lounge,Middle Eastern Restaurant,Multiplex,Music Venue,Nightclub,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Steakhouse,Supermarket,Tea Room,Vietnamese Restaurant
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
# @Hidden_cell

Abidjan_onehot.shape

(54, 34)